# Manual Testing

See `readme.md` for setup instructions.

Then make sure this jupyter notebook is restarted before rebuilding, or `maturin` will fail to copy the fresh `.pyd` library into `.env/Lib/megalinref/` .

In [3]:
import json
import megalinref as mlr
from pathlib import Path
import pandas as pd
import numpy as np
import util.unit_conversion as uc
import math

In [6]:
path_to_temp_data = Path(".").resolve().joinpath("temporary_test_data")
path_to_temp_data.mkdir(exist_ok=True)
path_to_temp_data /= "road_network.json"

if not path_to_temp_data.exists():
    print(f"There is no data cached at {path_to_temp_data}")
    # Fetch new data
    cached_json_data = mlr.download_fresh_data_as_json()
    # Cache the data
    with path_to_temp_data.open("w") as f:
        json.dump(cached_json_data, f)
else:
    # Use cached data
    with path_to_temp_data.open("r") as f:
        cached_json_data = json.load(f)

In [11]:
cways = set()
network_types = set()
for item in cached_json_data["features"]:
    cways.add(item["properties"]["CWY"])
    network_types.add(item["properties"]["NETWORK_TYPE"])
print(cways)
print(network_types)

{'Left', 'Single', 'Right'}
{'Local Road', 'Crossover', 'Proposed Road', 'Miscellaneous Road', 'Main Roads Controlled Path', 'State Road'}


In [7]:
cached_json_data["features"][1]

{'type': 'Feature',
 'geometry': {'type': 'LineString',
  'coordinates': [[116.01613363394955, -31.90864177309936],
   [116.01564263139306, -31.90885735305153]]},
 'properties': {'ROAD': 'X001',
  'START_SLK': 0.03,
  'END_SLK': 0.08,
  'CWY': 'Single',
  'NETWORK_TYPE': 'Crossover',
  'START_TRUE_DIST': 0.03,
  'END_TRUE_DIST': 0.08}}

In [8]:
lookup = mlr.Lookup.from_dict(cached_json_data)
lookup

In [7]:
bb = lookup.to_binary()

with open("./temporary_test_data/binary_cache_new.bin", "wb") as f:
    f.write(bb)

In [12]:
result = lookup.road_slk_from_coordinate(
    lat = -31.89006203575722,
    lon = 115.80183730752809,
    carriageways  = mlr.Cwy["All"],
    network_types = mlr.NetworkType["All"],
    roads = []
)

In [13]:
#[[(lon, lat)]] = 
lookup.coordinate_from_road_slk("H001", 1, mlr.Cwy["Left"] | mlr.Cwy["Single"])
#f"https://www.google.com.au/maps/@{lat},{lon},22z"

[[], [(115.8873090200249, -31.967405903345778)]]

In [14]:
import plotly.express as px
import pandas as pd

result = lookup.coordinate_from_road_slk("H001", 0.1, mlr.Cwy["Left"])
result2 = []
for list_of_tuples in result:
    result2.extend(list(item) for item in list_of_tuples)
df = pd.DataFrame(
     result2,
     columns=["lon","lat"]
)
df

ModuleNotFoundError: No module named 'plotly'

In [43]:
fig = px.scatter_mapbox(
    df,
    lat='lat',
    lon='lon'
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [4]:
result = slk_lookup.road_slk_from_coordinate(
    -32.0832347,
    115.8741531,
    mlr.Cwy["All"],
    mlr.NetworkType["All"]
)
result

PanicException: called `Option::unwrap()` on a `None` value

In [7]:
# should be Mitchell Fwy, left lane at slk 10
lat_lon = (-31.89006203575722, 115.80183730752809)
expected_result = {
    "feature":{
        "ROAD": "H016",
        "CWY": "Left",
    },
    "slk":10
}

result = slk_lookup.lookup(*lat_lon, any_carriageway, any_network_type)

assert all(result["feature"][item] == expected_result["feature"][item] for item in ["ROAD","CWY"])
assert math.isclose(result["slk"], expected_result["slk"], abs_tol=0.01)

In [14]:
# should be Mitchell Fwy, left lane at slk 10
lat_lon = (-31.89006203575722, 115.80183730752809)
expected_result = {
    "feature":{
        "ROAD": "H016",
        "CWY": "Left",
    },
    "slk":10
}

bincode = slk_lookup.to_binary()
with open("./temporary_test_data/binary_cache.bin", "wb") as f:
    f.write(bincode)

with open("./temporary_test_data/binary_cache.bin", "rb") as f:
    bincode = f.read()

slk_lookup_reloaded = mlr.SLKLookup.from_binary(bincode)
result = slk_lookup_reloaded.lookup(*lat_lon, any_carriageway, any_network_type)
assert all(result["feature"][item] == expected_result["feature"][item] for item in ["ROAD","CWY"])
assert math.isclose(result["slk"], expected_result["slk"], abs_tol=0.01)

In [2]:
with open("./temporary_test_data/binary_cache.bin", "rb") as f:
    bincode = f.read()

slk_lookup_reloaded = mlr.SLKLookup.from_binary(bincode)

In [8]:
slk_lookup.lookup(
  lat=-31.89006203575722,
  lon=115.80183730752809,
  cwy=mlr.Cwy["L"] | mlr.Cwy["R"],
  network_type=mlr.NetworkType["State Road"] | mlr.NetworkType["Local Road"] 
)

{'feature': {'ROAD': 'H016',
  'CWY': 'Left',
  'START_SLK': 9.84,
  'END_SLK': 10.68,
  'START_TRUE_DIST': 9.84,
  'END_TRUE_DIST': 10.68,
  'NETWORK_TYPE': 'State Road'},
 'slk': 9.99999981522603,
 'true': 9.99999981522603,
 'distance_metres': 1.1852499613658425e-09}

In [10]:
type(bb)

bytes

In [11]:
f"{len(bb)/1024/1024:.2f} MB"

'32.70 MB'

In [12]:
import json
f"{len(json.dumps(xx))/1024/1024:.2f} MB"

'98.85 MB'